In [ ]:
# !pip install fastapi uvicorn nest-asyncio


In [1]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile, HTTPException
from PIL import Image
import io
import tensorflow as tf
import numpy as np

# Permitir que FastAPI se ejecute en Jupyter
nest_asyncio.apply()

# Inicializar la aplicación de FastAPI
app = FastAPI()

# Cargar el modelo entrenado de Keras en formato HDF5 (.h5)
model = tf.keras.models.load_model('../Results/ensemble_model.h5')  # Asegúrate de que la ruta sea correcta

# Clases de tu modelo (asegúrate de que el orden es correcto)
class_names = ['Chinche salivosa', 'Clororis', 'Hoja sana', 'Roya naranja', 'Roya purpura']

@app.post("/predict/")
async def predict_image(file: UploadFile = File(...)):
    try:
        # Leer la imagen subida
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))

        # Convertir la imagen a RGB y redimensionarla
        image = image.convert("RGB")
        image = image.resize((256, 256))  # Asegúrate de que coincide con INPUT_SHAPE

        # Convertir la imagen a un array de NumPy y a float32
        img_array = np.array(image).astype(np.float32)

        # Normalizar los valores de la imagen si tu modelo lo requiere (ej. dividir por 255)
        img_array = img_array / 255.0

        # Añadir dimensión para el batch
        img_array = np.expand_dims(img_array, axis=0)

        # Realizar la predicción
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        class_name = class_names[predicted_class]
        confidence = float(np.max(predictions))  # Asegúrate de convertir a float

        # Imprimir para depuración
        print("Predicciones:", predictions)
        print("Índice de clase predicha:", predicted_class)
        print("Clase predicha:", class_name)
        print("Confianza:", confidence)

        return {"prediction": class_name, "confidence": confidence}
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error al procesar la imagen: {str(e)}")

# Iniciar el servidor de FastAPI
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8001)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'ensemble_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [31]:
import requests

# URL de la API (usa la URL pública generada por ngrok)
url = "https://0088-34-142-189-227.ngrok-free.app/predict/"

# Subir una imagen para predecir (ajusta la ruta de la imagen según tu ubicación)
with open("../Chinche.jpg", "rb") as image_file:
    response = requests.post(url, files={"file": image_file})

# Verificar si la respuesta es correcta antes de decodificar
if response.status_code == 200:
    try:
        json_response = response.json()  # Intentar decodificar JSON
        print(json_response)
    except ValueError as e:
        print("Error al decodificar JSON:", e)
        print("Contenido de la respuesta:", response.text)
else:
    print(f"Error en la solicitud: {response.status_code}")
    print("Contenido de la respuesta:", response.text)


{'prediction': 'Roya purpura', 'confidence': 1.0}
